In [1]:
%%capture
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/WhenPigsFlyContext/baselines/Detectron2

!python -m pip install pyyaml==5.1
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install ml-collections

In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random, pickle
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.utils.events import EventStorage
from detectron2.structures import Boxes, ImageList, Instances, pairwise_iou
import detectron2.data.transforms as T

# import pytorch
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import to_pil_image, to_tensor

from PIL import Image
from tqdm import tqdm, trange
%matplotlib inline
from matplotlib import pyplot as plt
import sys
sys.path.append("..")
import copy
from naturaldesign.naturaldesign import NaturalDesign
from utils import *

In [4]:
# transform attention map from rgb to grayscale
transform_single = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor()
    ]
) 

# change PIL image to pytorch tensor
img_transforms = transforms.Compose(
    [
        transforms.Resize((800, 1280)),
        transforms.ToTensor(),
    ]
)

# process predicted objectiveness logits to range(0, 1)
sigmoid = torch.nn.Sigmoid()

In [5]:
test_imagedir = "../../naturaldesign/"
dataset = NaturalDesign(test_imagedir)

In [6]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
aug = T.ResizeShortestEdge(
            [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST
        )

predictor = DefaultPredictor(cfg)
model = build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

model_final_f10217.pkl: 178MB [00:07, 23.5MB/s]                           


{'__author__': 'Detectron2 Model Zoo'}

In [ ]:
#@title Demo of Detectron2
im, _ = dataset[2]
# cv2_imshow(im)
outputs = predictor(im)
print(outputs["instances"].pred_classes)

# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

In [7]:
detectron2_res, size, img_size, original_size = [], 200, (1024, 1280), (1024, 1280)
detectron2_attention_map, scanpath = {}, {}

for id in trange(1, len(dataset)+1):

    img, bbox_relative = dataset[id]
    # get the target bounding box
    tg_loc = bbox_cordinates(bbox_relative, img_size[1], img_size[0])

    # get the img path
    image = aug.get_transform(img).apply_image(img)
    img_tensor = torch.as_tensor(image.astype("float32").transpose(2, 0, 1)).cuda()
    img_tensor = transforms.Resize(original_size)(img_tensor)
    imgList = ImageList.from_tensors([img_tensor])

    # compute the attention map
    model.eval()
    with torch.no_grad():
        features = model.backbone(img_tensor.unsqueeze(0))
        mask_features = [features[f] for f in features.keys()]
        pred_objectness_logits, _ = model.proposal_generator.rpn_head(mask_features)

    attention_map = torch.zeros((3, 1024, 1280)).detach().cuda()
    for i in range(len(pred_objectness_logits)):
        logits = transforms.Resize(img_size)(pred_objectness_logits[i].squeeze(0)).cuda()
        logits = sigmoid(logits)
        attention_map += logits

    attention_map = attention_map/len(pred_objectness_logits)
    attention_map = torch.divide(attention_map, attention_map.max())
    attention_map = transform_single(to_pil_image(attention_map))

    detectron2_attention_map[id] = copy.deepcopy(attention_map)

    search_num, path = searchProcesswithPath(tg_loc, attention_map, img_size, size)
    scanpath[id] = path
    print(search_num)
    detectron2_res.append(search_num)

  0%|          | 1/240 [00:07<29:04,  7.30s/it]

2


  1%|          | 2/240 [00:08<13:48,  3.48s/it]

57


  1%|▏         | 3/240 [00:09<09:26,  2.39s/it]

...

4


100%|█████████▉| 239/240 [02:05<00:00,  2.22it/s]

48


100%|██████████| 240/240 [02:05<00:00,  1.91it/s]

5


In [8]:
detectron2_naturaldesign_res = {}
detectron2_naturaldesign_res['scanpath'] = scanpath
detectron2_naturaldesign_res['attention_map'] = detectron2_attention_map

In [9]:
with open("../results/NaturalDesign/[Fig5]NaturalDesign_detectron2_res.pkl", "wb") as tf:
    pickle.dump(detectron2_naturaldesign_res, tf)

In [ ]:
detectron2_accu_ND = model_performance(detectron2_res, len(detectron2_res))
detectron2_accu_ND[:11]

[0,
 0.1125,
 0.25416666666666665,
 0.325,
 0.375,
 0.4166666666666667,
 0.4583333333333333,
 0.5208333333333334,
 0.5583333333333333,
 0.5958333333333333,
 0.6166666666666667]

In [ ]:
with open("../results/NaturalDesign/naturaldesign_detectron2_accu_performance.pkl", "wb") as tf:
    pickle.dump(detectron2_accu_ND,tf)